In [37]:
# Import modules
import arcpy, os
import re

##### Prepare necessary parameters

In [3]:
arcpy.env.workspace = '../Step_1_Data_forcing_01yr_010deg/'
nc_files = arcpy.ListFiles()

##### Create raster layers from nc files

In [4]:
# define the path to nc files
nc_path = '../Step_1_Data_forcing_01yr_010deg'

# innitiate a layer dict to store ne_tif layers
Nc_layr = {}

# allow overwrrting 
arcpy.env.overwriteOutput = True

# create raster layers from nc
for nc in nc_files:
    
    nc_name = nc[:4]

    Nc_layr[f"{nc_name}"] = arcpy.MakeNetCDFRasterLayer_md(in_netCDF_file=f"{nc_path}/{nc}", 
                                                           variable=f"{nc_name}", 
                                                           x_dimension="lon", 
                                                           y_dimension="lat", 
                                                           out_raster_layer=f"{nc_name}_Layer", 
                                                           band_dimension="time", 
                                                           dimension_values="", 
                                                           value_selection_method="BY_VALUE")
    

##### Export each-year band

In [28]:
# define the path to nc files
out_path = '../Step_2_Per_year_tifs'

# export each band from the nc_layers to single band img
for name,lyr in Nc_layr.items():

    #Reading number of band information from saved TIF
    bandcount = arcpy.GetRasterProperties_management (lyr, "BANDCOUNT") 
    resultValue = bandcount.getOutput(0)
    
    # copy lyr to dirve so we can select individul band
    arcpy.CopyRaster_management(in_raster = lyr, 
                                out_rasterdataset=os.path.abspath(f'{out_path}/{name}.tif'))
    start_year = 1979
    count = 1
    #Loop through the bands and copy bands as a seperate TIF file.
    while count <= int(resultValue):
        
        # copy each band to drive
        Input_Raster = os.path.abspath(f'{out_path}/{name}.tif/Band_{str(count)}')
        Output_Band  = os.path.abspath(f'{out_path} /{name}_{str(start_year)}.tif')
        
        arcpy.CopyRaster_management(Input_Raster, Output_Band)
        
        # print out the process
        print(f'Exporting {name}_{str(start_year)}.tif sucessful')
        
        # update flags
        count +=1
        start_year +=1
    
    #  delete the multibands nc tif  
    arcpy.Delete_management(os.path.abspath(f'{out_path}/{name}.tif'),"#")


Exporting lrad_1979.tif sucessful
Exporting lrad_1980.tif sucessful
Exporting lrad_1981.tif sucessful
Exporting lrad_1982.tif sucessful
Exporting lrad_1983.tif sucessful
Exporting lrad_1984.tif sucessful
Exporting lrad_1985.tif sucessful
Exporting lrad_1986.tif sucessful
Exporting lrad_1987.tif sucessful
Exporting lrad_1988.tif sucessful
Exporting lrad_1989.tif sucessful
Exporting lrad_1990.tif sucessful
Exporting lrad_1991.tif sucessful
Exporting lrad_1992.tif sucessful
Exporting lrad_1993.tif sucessful
Exporting lrad_1994.tif sucessful
Exporting lrad_1995.tif sucessful
Exporting lrad_1996.tif sucessful
Exporting lrad_1997.tif sucessful
Exporting lrad_1998.tif sucessful
Exporting lrad_1999.tif sucessful
Exporting lrad_2000.tif sucessful
Exporting lrad_2001.tif sucessful
Exporting lrad_2002.tif sucessful
Exporting lrad_2003.tif sucessful
Exporting lrad_2004.tif sucessful
Exporting lrad_2005.tif sucessful
Exporting lrad_2006.tif sucessful
Exporting lrad_2007.tif sucessful
Exporting lrad

Exporting wind_1980.tif sucessful
Exporting wind_1981.tif sucessful
Exporting wind_1982.tif sucessful
Exporting wind_1983.tif sucessful
Exporting wind_1984.tif sucessful
Exporting wind_1985.tif sucessful
Exporting wind_1986.tif sucessful
Exporting wind_1987.tif sucessful
Exporting wind_1988.tif sucessful
Exporting wind_1989.tif sucessful
Exporting wind_1990.tif sucessful
Exporting wind_1991.tif sucessful
Exporting wind_1992.tif sucessful
Exporting wind_1993.tif sucessful
Exporting wind_1994.tif sucessful
Exporting wind_1995.tif sucessful
Exporting wind_1996.tif sucessful
Exporting wind_1997.tif sucessful
Exporting wind_1998.tif sucessful
Exporting wind_1999.tif sucessful
Exporting wind_2000.tif sucessful
Exporting wind_2001.tif sucessful
Exporting wind_2002.tif sucessful
Exporting wind_2003.tif sucessful
Exporting wind_2004.tif sucessful
Exporting wind_2005.tif sucessful
Exporting wind_2006.tif sucessful
Exporting wind_2007.tif sucessful
Exporting wind_2008.tif sucessful
Exporting wind

##### compute the mean of each-year band img

In [46]:
# get the each-year band img
arcpy.env.workspace = '../Step_2_Per_year_tifs/'

# get the unique name of all year_imgs
name = list(set([i[:4] for i in arcpy.ListRasters('*.tif')]))

for n in name:
    
    year_img =  arcpy.ListRasters(f'{n}*')
    
    # compute the sum img
    i = 0
    for img in year_img:
        if i == 0:
            sum_img = img
        else:
            sum_img = arcpy.sa.Raster(sum_img) + arcpy.sa.Raster(img)
        i += 1
            
    Mean_img =  arcpy.sa.Raster(sum_img)/len(year_img)
    Mean_img.save(f'../Step_3_Mean_tifs/{n}.tif')
    